# Demo: Распознавание рукописного кириллического текста

**Проект:** Handwritten OCR для русского текста  
**Модель:** TrOCR Raxtemur (лучшая по результатам тестирования)  
**CER:** 54.7% (800+ экспериментов)  

## О проекте

Этот ноутбук демонстрирует работу оптимальной модели **TrOCR Raxtemur** для распознавания рукописного русского текста.

**Результаты тестирования:**
- Протестировано 3 модели на 200+ изображениях
- 800+ экспериментов с различными аугментациями
- TrOCR Raxtemur показал наиболее стабильный результат: **CER 54.7%**


## 1. Установка зависимостей

Запустите эту ячейку один раз для установки необходимых библиотек.

In [ ]:
# Установка зависимостей
!pip install -q transformers torch pillow paddleocr python-Levenshtein cer werpy

print("Зависимости установлены")

## 2. Импорты

In [ ]:
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from paddleocr import PaddleOCR
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
from cer import calculate_cer
from werpy import wer
import warnings
warnings.filterwarnings('ignore')

print("Библиотеки импортированы")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 3. Загрузка модели TrOCR Raxtemur

**TrOCR Raxtemur** - лучшая модель по результатам 800+ экспериментов.

**Характеристики:**
- **CER:** 54.7% (наиболее стабильный результат)
- **Архитектура:** TrOCR base
- **Обучение:** Синтетические данные HKR/Cyrillic
- **Время обработки:** 650-850 мс на изображение
- **Специализация:** Современный русский рукописный текст

In [ ]:
print("Загрузка модели TrOCR Raxtemur...")

# Определяем устройство
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используется устройство: {device}")

# Загрузка модели
model_name = 'raxtemur/trocr-base-handwritten-cyrillic'
processor = TrOCRProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)
model.to(device)
model.eval()

print("\n Модель загружена успешно")
print(f"Модель: {model_name}")
print(f"Устройство: {device}")
print(f"Протестирована на 200+ изображениях (800+ экспериментов)")
print(f"CER: 54.7%")

## 4. Инициализация детектора строк

PaddleOCR используется для автоматической детекции областей текста.

In [ ]:
print("Инициализация PaddleOCR...")
ocr_detector = PaddleOCR(use_angle_cls=False, lang='ru', show_log=False)
print("Детектор инициализирован")

## 5. Функция распознавания с визуализацией

In [ ]:
def recognize_text(image_path, ground_truth=None, show_detection=True):
    """
    Распознает рукописный текст на изображении.
    
    Args:
        image_path: Путь к изображению или PIL.Image
        ground_truth: Эталонный текст (опционально)
        show_detection: Показывать детекцию строк
    
    Returns:
        recognized_text: Распознанный текст
    """
    # Загрузка изображения
    if isinstance(image_path, str):
        image = Image.open(image_path).convert('RGB')
    else:
        image = image_path
    
    # Детекция строк
    result = ocr_detector.ocr(np.array(image), rec=False)
    
    if result is None or result[0] is None:
        print("Строки не обнаружены")
        return ""
    
    boxes = result[0]
    print(f"Обнаружено строк: {len(boxes)}")
    
    # Визуализация детекции
    if show_detection:
        fig, ax = plt.subplots(1, 1, figsize=(12, 8))
        ax.imshow(image)
        
        for box in boxes:
            box = np.array(box).reshape((-1, 2))
            ax.plot(box[:, 0], box[:, 1], 'r-', linewidth=2)
            ax.plot([box[-1, 0], box[0, 0]], [box[-1, 1], box[0, 1]], 'r-', linewidth=2)
        
        ax.set_title(f'Детекция строк: {len(boxes)} найдено', fontsize=14)
        ax.axis('off')
        plt.tight_layout()
        plt.show()
    
    # Распознавание каждой строки
    recognized_lines = []
    
    for idx, box in enumerate(boxes):
        # Вырезаем строку
        box = np.array(box).reshape((-1, 2)).astype(int)
        x_min, y_min = box.min(axis=0)
        x_max, y_max = box.max(axis=0)
        
        line_img = image.crop((x_min, y_min, x_max, y_max))
        
        # Распознаем строку
        pixel_values = processor(line_img, return_tensors="pt").pixel_values.to(device)
        generated_ids = model.generate(pixel_values)
        text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        
        recognized_lines.append(text)
    
    # Объединяем строки
    recognized_text = ' '.join(recognized_lines)
    
    # Вывод результатов
    print("\n" + "="*60)
    print("РЕЗУЛЬТАТ РАСПОЗНАВАНИЯ:")
    print("="*60)
    print(recognized_text)
    print("="*60)
    
    # Метрики качества
    if ground_truth:
        cer_value = calculate_cer(ground_truth, recognized_text)
        wer_value = wer(ground_truth, recognized_text)
        
        print("\n МЕТРИКИ КАЧЕСТВА:")
        print(f"CER (Character Error Rate): {cer_value:.2%}")
        print(f"WER (Word Error Rate): {wer_value:.2%}")
        print(f"Character Accuracy: {(1-cer_value):.2%}")
        print("\n ЭТАЛОННЫЙ ТЕКСТ:")
        print(ground_truth)
    
    return recognized_text

print("Функция распознавания готова")

## 6. Демонстрация

### Пример 1: Загрузите свое изображение

Для Google Colab используйте:

In [ ]:
# Загрузка изображения в Google Colab
from google.colab import files
uploaded = files.upload()

# Берем первый загруженный файл
image_path = list(uploaded.keys())[0]
print(f"Загружен файл: {image_path}")

# Опционально: укажите эталонный текст для расчета метрик
ground_truth = None  # Или укажите: "ваш эталонный текст"

# Распознаем
text = recognize_text(image_path, ground_truth=ground_truth)

### Пример 2: Тестирование на примере из датасета

Если у вас есть доступ к тестовым данным проекта:

In [ ]:
# Пример использования с локальными данными
# Замените путь на ваш

# image_path = 'data/processed/hwr200/tenPercentTest/hwr200_01_03_01_7.JPG'
# ground_truth = "ваш эталонный текст"

# text = recognize_text(image_path, ground_truth=ground_truth)

print("Раскомментируйте код выше и укажите путь к изображению")

### Пример 3: Обработка нескольких изображений

Для массовой обработки изображений:

In [ ]:
def process_folder(folder_path, output_file='results.txt'):
    """
    Обрабатывает все изображения в папке.
    
    Args:
        folder_path: Путь к папке с изображениями
        output_file: Файл для сохранения результатов
    """
    import os
    
    results = []
    
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            print(f"\nОбработка: {filename}")
            
            try:
                text = recognize_text(image_path, show_detection=False)
                results.append(f"{filename}: {text}")
            except Exception as e:
                print(f"Ошибка при обработке {filename}: {e}")
                results.append(f"{filename}: ERROR")
    
    # Сохраняем результаты
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(results))
    
    print(f"\n Результаты сохранены в {output_file}")
    return results

# Пример использования:
# results = process_folder('data/test_images/')

print(" Функция batch processing готова")